In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# read data into memory
data_set_train = np.genfromtxt("hw04_data_set_train.csv", delimiter = ",", skip_header = 1)
data_set_test = np.genfromtxt("hw04_data_set_test.csv", delimiter = ",", skip_header = 1)

# get x and y values
X_train = data_set_train[:, 0:1]
y_train = data_set_train[:, 1]
X_test = data_set_test[:, 0:1]
y_test = data_set_test[:, 1]

# set drawing parameters
minimum_value = 1.5
maximum_value = 5.1
step_size = 0.001
X_interval = np.arange(start = minimum_value, stop = maximum_value + step_size, step = step_size)
X_interval = X_interval.reshape(len(X_interval), 1)

In [3]:
def plot_figure(X_train, y_train, X_test, y_test, X_interval, y_interval_hat):
    fig = plt.figure(figsize = (8, 4))
    plt.plot(X_train[:, 0], y_train, "b.", markersize = 10)
    plt.plot(X_test[:, 0], y_test, "r.", markersize = 10)
    plt.plot(X_interval[:, 0], y_interval_hat, "k-")
    plt.xlabel("Eruption time (min)")
    plt.ylabel("Waiting time to next eruption (min)")
    plt.legend(["training", "test"])
    plt.show()
    return(fig)

In [4]:
# STEP 2
# should return necessary data structures for trained tree
def decision_tree_regression_train(X_train, y_train, P):
    # create necessary data structures
    node_indices = {}
    is_terminal = {}
    need_split = {}


    node_features = {}
    node_splits = {}
    node_means = {} # Instead of frequencies, we store the mean of y values in each node
    # your implementation starts below

    # Initialization
    node_indices = {0: np.arange(X_train.shape[0])}
    is_terminal = {0: False}
    need_split = {0: True}

    while True:
        split_nodes = [node for node, need in need_split.items() if need]
        if not split_nodes:
            break

        for node in split_nodes:
            data_indices = node_indices[node]
            if len(data_indices) <= P:
                is_terminal[node] = True
                need_split[node] = False
                node_means[node] = np.mean(y_train[data_indices]) if data_indices.size > 0 else 0
                continue

            best_feature, best_split, min_mse = None, None, float("inf")
            for feature in range(X_train.shape[1]):
                unique_splits = np.unique(X_train[data_indices, feature])
                for split in unique_splits:
                    left_indices = data_indices[X_train[data_indices, feature] < split]
                    right_indices = data_indices[X_train[data_indices, feature] >= split]
                    
                    # Only attempt to split if both left and right have more than zero elements
                    if len(left_indices) > 0 and len(right_indices) > 0:
                        left_mean = np.mean(y_train[left_indices])
                        right_mean = np.mean(y_train[right_indices])
                        
                        left_mse = np.mean((y_train[left_indices] - left_mean) ** 2)
                        right_mse = np.mean((y_train[right_indices] - right_mean) ** 2)
                        
                        mse = (len(left_indices) * left_mse + len(right_indices) * right_mse) / len(data_indices)
                        
                        if mse < min_mse:
                            min_mse = mse
                            best_feature = feature
                            best_split = split
                unique_splits = np.unique(X_train[data_indices, feature])
                for split in unique_splits:
                    left_indices = data_indices[X_train[data_indices, feature] < split]
                    right_indices = data_indices[X_train[data_indices, feature] >= split]


            if best_feature is not None:
                left_indices = data_indices[X_train[data_indices, best_feature] < best_split]
                right_indices = data_indices[X_train[data_indices, best_feature] >= best_split]

                left_node = len(node_indices)
                right_node = left_node + 1

                node_indices[left_node] = left_indices
                node_indices[right_node] = right_indices
                is_terminal[node] = False
                need_split[node] = False
                is_terminal[left_node] = False
                need_split[left_node] = True
                is_terminal[right_node] = False
                need_split[right_node] = True
                node_features[node] = best_feature
                node_splits[node] = best_split
                node_means[node] = np.mean(y_train[data_indices])

            else:
                is_terminal[node] = True
                need_split[node] = False
                node_means[node] = np.mean(y_train[data_indices]) if data_indices.size > 0 else 0

    # your implementation ends above
    return(is_terminal, node_features, node_splits, node_means)

In [5]:
decision_tree_regression_train(X_train, y_train, 25)[3]

{0: 70.73333333333333,
 1: 54.75,
 2: 80.25531914893617,
 3: 53.86363636363637,
 4: 58.0,
 5: 76.92105263157895,
 6: 82.51785714285714,
 7: 58.333333333333336,
 8: 53.53658536585366,
 9: 73.0,
 10: 77.38235294117646,
 11: 82.27272727272727,
 12: 96.0,
 13: 52.57692307692308,
 14: 55.2,
 15: 81.5,
 16: 76.5,
 17: 84.5,
 18: 81.77777777777777,
 19: 54.1764705882353,
 20: 49.55555555555556,
 21: 74.0,
 22: 77.18181818181819,
 23: 80.1875,
 24: 82.65517241379311,
 25: 82.89285714285714,
 26: 76.0,
 27: 82.13636363636364,
 28: 85.66666666666667}

In [6]:
def decision_tree_get_y_hat(x, is_terminal, node_features, node_splits, node_means):
    node = 0
    while not is_terminal[node]:  # Continue until a terminal node is reached

        split = node_splits[node]

        if x < split:
            node = 2 * node + 1  # Go to left child
        else:
            node = 2 * node + 2  # Go to right child

    return node_means[node]


In [7]:
# STEP 3
# assuming that there are N query data points
# should return a numpy array with shape (N,)
def decision_tree_regression_test(X_query, is_terminal, node_features, node_splits, node_means):
    # your implementation starts below
    y_hat = np.zeros(X_query.shape[0])
    for i, x in enumerate(X_query):
        y_hat[i] = decision_tree_get_y_hat(x, is_terminal, node_features, node_splits, node_means)
    # your implementation ends above
    return(y_hat)

In [8]:
is_terminal, node_features, node_splits, node_means = decision_tree_regression_train(X_train, y_train, 25)
decision_tree_regression_test(np.array([1, 3.5, 6]), is_terminal, node_features, node_splits, node_means)

array([58.33333333, 96.        , 55.2       ])

In [9]:
# STEP 4
# assuming that there are T terminal node
# should print T rule sets as described
def extract_rule_sets(is_terminal, node_features, node_splits, node_means):
    # your implementation starts below
    
    # your implementation ends above

SyntaxError: incomplete input (2207379865.py, line 7)

In [ ]:
P = 20
is_terminal, node_features, node_splits, node_means = decision_tree_regression_train(X_train, y_train, P)
y_interval_hat = decision_tree_regression_test(X_interval, is_terminal, node_features, node_splits, node_means)
fig = plot_figure(X_train, y_train, X_test, y_test, X_interval, y_interval_hat)
fig.savefig("decision_tree_regression_{}.pdf".format(P), bbox_inches = "tight")

y_train_hat = decision_tree_regression_test(X_train, is_terminal, node_features, node_splits, node_means)
rmse = np.sqrt(np.mean((y_train - y_train_hat)**2))
print("RMSE on training set is {} when P is {}".format(rmse, P))

y_test_hat = decision_tree_regression_test(X_test, is_terminal, node_features, node_splits, node_means)
rmse = np.sqrt(np.mean((y_test - y_test_hat)**2))
print("RMSE on test set is {} when P is {}".format(rmse, P))

P = 50
is_terminal, node_features, node_splits, node_means = decision_tree_regression_train(X_train, y_train, P)
y_interval_hat = decision_tree_regression_test(X_interval, is_terminal, node_features, node_splits, node_means)
fig = plot_figure(X_train, y_train, X_test, y_test, X_interval, y_interval_hat)
fig.savefig("decision_tree_regression_{}.pdf".format(P), bbox_inches = "tight")

y_train_hat = decision_tree_regression_test(X_train, is_terminal, node_features, node_splits, node_means)
rmse = np.sqrt(np.mean((y_train - y_train_hat)**2))
print("RMSE on training set is {} when P is {}".format(rmse, P))

y_test_hat = decision_tree_regression_test(X_test, is_terminal, node_features, node_splits, node_means)
rmse = np.sqrt(np.mean((y_test - y_test_hat)**2))
print("RMSE on test set is {} when P is {}".format(rmse, P))

extract_rule_sets(is_terminal, node_features, node_splits, node_means)


KeyError: 37